### Recommendations with MovieTweetings: Most Popular Recommendation

Now that you have created the necessary columns we will be using throughout the rest of the lesson on creating recommendations, let's get started with the first of our recommendations.

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

In [2]:
reviews.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,2,454876,8,1394818630,2014-03-14 17:37:10,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2,790636,7,1389963947,2014-01-17 13:05:47,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [3]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [4]:
def popular_recommendations(user_id, n_top):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    grouped = reviews.groupby('movie_id').agg(
    {'rating': [('ratings', 'count'),
                ('avg_rating', 'mean')],
     'date':[('rated_last','max')]
    })
    grouped.columns=grouped.columns.droplevel()
    grouped = grouped.sort_values(by=['avg_rating','ratings','rated_last'],ascending=False)
    grouped = grouped.loc[grouped['ratings']>5]
    grouped = grouped.merge(movies[['movie_id','movie']],left_index=True, right_on=['movie_id'])
    top_movies = grouped[:n_top]['movie'].tolist()
    
    return top_movies # a list of the n_top movies as recommended

In [5]:
popular_recommendations(1, 5)

['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Sorry to Bother You (2018)',
 'Selam (2013)',
 "Quiet Riot: Well Now You're Here, There's No Way Back (2014)"]

**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
```

In [6]:
def recommend_filtered(user_id, n_top, years,genres):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    grouped = reviews.groupby('movie_id').agg(
    {'rating': [('ratings', 'count'),
                ('avg_rating', 'mean')],
     'date':[('rated_last','max')]
    })
    grouped.columns=grouped.columns.droplevel()
    grouped = grouped.sort_values(by=['avg_rating','ratings','rated_last'],ascending=False)
    grouped = grouped.loc[grouped['ratings']>5]
    grouped = grouped.merge(movies,left_index=True, right_on=['movie_id'])
    if years:
        grouped = grouped.loc[grouped['date'].isin(years)]

    if genres:
        grouped = grouped.loc[(grouped[genres] == 1).any(axis=1)]
    
    return grouped # a list of the n_top movies as recommended

In [7]:
recommend_filtered('1', 20, years=['2017', '2018'], genres=['History'])

,ratings,avg_rating,rated_last,movie_id,movie,genre,date,1800's,1900's,2000's,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
30587,35,9.200000,2018-06-22 22:19:09,6316138,Ayla: The Daughter of War (2017),Drama|History|War,2017,0,0,1,...,0,0,0,0,0,0,0,0,0,0
30503,6,8.833333,2018-06-11 04:06:25,6223974,The Farthest (2017),Documentary|History,2017,0,0,1,...,0,0,0,0,1,0,0,0,0,0
30862,6,8.333333,2018-03-18 13:27:52,6794424,LA 92 (2017),Documentary|History,2017,0,0,1,...,0,0,0,0,1,0,0,0,0,0
28997,1618,7.982695,2018-06-25 02:12:19,5013056,Dunkirk (2017),Action|Drama|History,2017,0,0,1,...,0,0,0,1,0,0,0,0,0,0
30891,64,7.953125,2018-06-22 22:48:04,6878038,Taeksi Woonjunsa (2017),Action|Drama|History,2017,0,0,1,...,0,0,0,1,0,0,0,0,0,0
30220,18,7.944444,2018-06-24 13:06:26,5969696,The Battleship Island (2017),Action|Drama|History,2017,0,0,1,...,0,0,0,1,0,0,0,0,0,0
28273,487,7.895277,2018-06-25 02:47:00,4555426,Darkest Hour (2017),Biography|Drama|History,2017,0,0,1,...,0,0,0,0,0,0,0,0,0,0
28946,11,7.363636,2018-05-29 17:22:48,4977530,Viceroy's House (2017),Drama|History,2017,0,0,1,...,0,0,0,0,0,0,0,0,0,0
30569,320,7.331250,2018-06-23 11:31:29,6294822,The Post (2017),Biography|Drama|History,2017,0,0,1,...,0,0,0,0,0,0,0,0,0,0
30063,78,7.256410,2018-05-27 21:41:50,5816682,Victoria and Abdul (2017),Biography|Drama|History,2017,0,0,1,...,0,0,0,0,0,0,0,0,0,0
